In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from analysis.modules.moving_average import moving_average_analysis

In [3]:
investment_preference = {
    (5, 10) : 0.5,
    (10, 20) : 0.3,
    (20, 50) : 0.2,
}

In [4]:
df = moving_average_analysis('ACB', investment_preference)
df.tail()

,stock_exchange,stock_code,date,open_price,highest_price,lowest_price,close_price,volume,ma_5,ma_10,ma_20,ma_50,state_5_10,state_10_20,state_20_50
177,HSX,ACB,2021-08-31,32.5,32.60,32.00,32.00,5528600,32.06,32.905,34.3450,33.7722,wait,wait,wait
178,HSX,ACB,2021-09-01,32.0,32.75,31.95,32.00,8715300,31.97,32.550,34.1700,33.9602,wait,wait,wait
179,HSX,ACB,2021-09-06,32.2,32.30,31.95,32.05,7383200,31.99,32.225,33.9725,34.1532,wait,wait,bull
180,HSX,ACB,2021-09-07,32.4,32.75,32.20,32.50,7963700,32.14,32.115,33.8200,34.3520,bear,wait,wait
181,HSX,ACB,2021-09-08,32.5,32.75,32.10,32.15,4752900,32.14,32.130,33.6025,34.2610,wait,wait,wait
